<a href="https://colab.research.google.com/github/Sug-ar-N-Spice/Dr.Chats/blob/Ty/New_Q%26A_Med_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 !pip install sentence-transformers 
 ! pip install sacremoses
 ! pip install transformers
 ! pip install datasets
 ! pip install torch
 ! pip install spacy
#!pip install -q gradio

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/59.7 kB ? eta -:--:--
     ---------------------------------------- 59.7/59.7 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/11.8 MB 3.0 MB/s eta 0:00:04
    --------------------------------------- 0.2/11.8 MB 2.8 MB/s eta 0:00:05
   - -------------------------------------- 0.5/11.8 MB 4.2 MB/s eta 0:00:03
   --- ------------------------------------ 1.0/11.8 MB 5.3 MB/s eta 0:00:03
   ----- ---------------------------------- 1.5/11.8 MB 6.9 MB/s eta 0:00:02
   

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.17.0 requires numpy<2.0.0,>=1.26.0; python_version >= "3.12", but you have numpy 2.0.2 which is incompatible.
contourpy 1.2.0 requires numpy<2.0,>=1.20, but you have numpy 2.0.2 which is incompatible.
numba 0.59.1 requires numpy<1.27,>=1.22, but you have numpy 2.0.2 which is incompatible.
pywavelets 1.5.0 requires numpy<2.0,>=1.22.4, but you have numpy 2.0.2 which is incompatible.
streamlit 1.32.0 requires numpy<2,>=1.19.3, but you have numpy 2.0.2 which is incompatible.


In [2]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM
from typing import List, Dict
import nltk
from nltk.corpus import stopwords
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
import spacy
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sentence_transformers import SentenceTransformer
from DrChatbot import DrChatbot

c:\Users\jerom\.conda\envs\drchats\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
def process_large_csv(file_path, window_size=1000, stride=500, batch_size=32):
    chatbot = DrChatbot(window_size=window_size, stride=stride, batch_size=batch_size)
    all_contexts = []

    # Process the CSV file in chunks to handle large files
    for chunk in pd.read_csv(file_path, chunksize=window_size, sep='\t'):
        contexts = chatbot.preprocess_data(chunk)
        all_contexts.extend(contexts)

    chatbot.fit_context_selector(all_contexts)
    return chatbot



In [12]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# Example usage
file_path = './Resources/ai_medical_dataset_train.csv'
chatbot = process_large_csv(file_path, window_size=1000, stride=500, batch_size=32)

Using device: cuda


C:\Users\jerom\AppData\Roaming\Python\Python312\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [ ]:
chatbot.save_model('ai_medical_chatbot.model')

In [ ]:


# Ask a question
question = "What are the symptoms of asthma?"
answer = chatbot.answer_question(question)
print(f"Q: {question}\nA: {answer}")

In [ ]:
# Process the large CSV file
chatbot = process_large_csv('processed_df_1.csv', window_size=1000, stride=500, batch_size=32)


In [ ]:
# Example usage
question = "what are the symptons of asthma?"
answer = chatbot.answer_question(question)
print(f"Q: {question}\nA: {answer}")

Q: what are the symptons of asthma?
A: Based on the following medical information: name group a member asthma asthma impact asthma exacerbations asthma impact asthma exacerbations asthma triggers on asthma-related quality life in patients severe or difficult-to-treat asthma goal national asthma guidelines background maintenance asthma control time a clear goal national asthma guidelines studies addressed natural history asthma control time objective to assess impairment domain asthma control time in patients persistent asthma to determine predictors consequences controlled uncontrolled asthma time methods patients 18-56 years old persistent asthma completed baseline november 2007 follow-up asthma surveys april july october 2008 included in study survey included asthma control test as well as questions regarding patient asthma characteristics health care utilization pharmacy exacerbations 2008 obtained administrative data results baseline first follow-up surveys completed by 1267 patien